In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from util import *

In [2]:
# from sklearn import set_config
# set_config(display='diagram') 

In [3]:
training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")

In [4]:
testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")

In [5]:
subset = list(set(tier1)-set(["PersonalizedProduct"]))
subset

['Imaging',
 'AnalysisAndModeling',
 'AnatomicalTarget',
 'CognitiveComputingAndLearningSystems',
 'Validation',
 'Manufacturing',
 'Verification',
 'SurgicalTracking',
 'DNAMapping/GeneticEngineering',
 'DataAcquisition',
 'SpecificationofUse',
 'SurgicalMethod',
 'SurgicalRobotics']

In [6]:
training_labels = training_set[subset]

In [7]:
testing_labels = testing_set[subset]

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [10]:
set(training_set.columns) - set(all_tiers)

{'abstract',
 'all_tiers',
 'all_tiers_100',
 'application_kind',
 'application_number',
 'application_number_formatted',
 'assignees',
 'citations',
 'cited_by',
 'claims',
 'country_code_x',
 'country_code_y',
 'cpc_codes',
 'description',
 'embedding_v1',
 'examiners',
 'family_id',
 'fi_codes',
 'filing_date',
 'fterm_codes',
 'gpa_number',
 'grant_date',
 'inventors',
 'kind',
 'kind_code',
 'padded_serial',
 'pct_number',
 'priority_date',
 'publication_date',
 'publication_number',
 'serial_x',
 'serial_y',
 'similar_npl',
 'similar_patents',
 'tier1_tags',
 'tier1_tier2_tags',
 'tier2_100',
 'tier3_100',
 'title',
 'top_terms',
 'url',
 'uspc_codes'}

In [11]:
training_set.top_terms

0      [prosthesis, member, coupling portion, portion...
1      [femur, tibia, implant, leg, patient, portion,...
2      [lateral, femur, patient, body, medial, slot, ...
3      [glenoid, virtual, alignment pin, patient, sca...
4      [bone, surface, patient, resection, jig, use, ...
                             ...                        
967    [method, bone, medial, patient, template, late...
968    [cutting guide, surgical cutting, customized s...
969    [patient, adapted, surface, surface model, imp...
970    [polymer, monomers, modulus, thiol, multifunct...
971    [implant, patient, peg, portion, bone, porous,...
Name: top_terms, Length: 972, dtype: object

In [12]:
cpc_embeddings = np.fromfile("/home/martin/patentmark/cpc.node2vec.emb.32d.bin", dtype=np.float32).reshape((-1,32))

import joblib
cpc_labelizer = joblib.load('./node2id.joblib')
cpc_lookup = {c: n for n, c in enumerate(cpc_labelizer.classes_)}

@f.collecting
def convert_cpc_codes(codes):
    for code in codes:
        if code in cpc_lookup:
            yield cpc_lookup[code]
    
def embed_cpc_codes(codes):
    embedding = np.zeros(32)
    converted = convert_cpc_codes(codes)
    
    if not converted:
        return embedding
    
    for code_id in converted:
        embedding = embedding + cpc_embeddings[code_id]
        
    return embedding / len(converted)

training_set['embedded_cpc'] = training_set.cpc_codes.apply(embed_cpc_codes)
training_set.embedded_cpc

testing_set['embedded_cpc'] = testing_set.cpc_codes.apply(embed_cpc_codes)
testing_set.embedded_cpc

0      [0.09129103335241477, -0.8074875394503276, -0....
1      [-0.0626441298850945, -0.8264780470303127, -0....
2      [-0.2087969978650411, -0.8326806823412577, -0....
3      [0.020394775830209256, -0.8215901732444764, -0...
4      [-0.26043402403593063, -0.6891247034072876, -0...
                             ...                        
238    [-0.23802674313386282, -0.628900408744812, -0....
239    [-0.3754243354002635, -0.6894144614537557, -0....
240    [-0.12913421913981438, -0.6960149183869362, -0...
241    [-0.3880331997688, -0.702021429171929, -0.2717...
242    [-0.2977850042283535, -0.7015813589096069, -0....
Name: embedded_cpc, Length: 243, dtype: object

In [18]:
np.array(testing_set.embedded_cpc.values.tolist()).shape

(243, 32)

In [133]:
from sklearn.ensemble import *

In [134]:
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier

In [135]:
from sklearn.dummy import DummyClassifier

In [136]:
from sklearn.linear_model import LogisticRegression

In [31]:
class VectorPassthrough(sklearn.base.BaseEstimator):
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return np.array(X.values.tolist())
        

In [37]:
tfidf_default_settings = {
    'lowercase': True, 
    'strip_accents': 'ascii',
    'stop_words' : stopwords,
    'min_df': 5,
    #'max_df': 0.5#,
    #'ngram_range': (1,3)
}

transformer = ColumnTransformer([
     ('top_terms',
      CountVectorizer(analyzer=iden, binary=True, min_df=2),
     'top_terms'
     ),
     ('cited_by',
      CountVectorizer(analyzer=iden, binary=True, min_df=2),
     'cited_by'
     ),
    ('inventors',
      CountVectorizer(analyzer=iden, binary=True, min_df=2),
     'inventors'
     ),
     ('citations',
       CountVectorizer(analyzer=iden, binary=True, min_df=2),
      'citations'
      ),
#         ('similar_npl',
#       CountVectorizer(analyzer=lambda x:x, min_df=2),
#      'similar_npl'
#      ),
        ('similar_patents',
      CountVectorizer(analyzer=iden, binary=True, min_df=2),
     'similar_patents'
     )
  ,
     ('cpc',
      CountVectorizer(analyzer=cpc_split, binary=True, min_df=2),
     'cpc_codes'
     ),
    ('embedded_cpc', 
     VectorPassthrough(),
     'embedded_cpc'
    ),
    ('embedding_v1',
     VectorPassthrough(),
     'embedding_v1'
    ),
    ('abstract_tfidf', 
    TfidfVectorizer(**tfidf_default_settings),
   'abstract'),
    ('claims_tfidf',
     TfidfVectorizer(**tfidf_default_settings),
     'claims'
    ),
#     ('description_tfidf',
#      TfidfVectorizer(**tfidf_default_settings),
#      'description'
#     )
      ], verbose=False, n_jobs=-1)

In [38]:
from sklearn.decomposition import TruncatedSVD

In [39]:
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.naive_bayes import GaussianNB

In [40]:
from sklearn.metrics import *

In [41]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
import util
X_train = transformer.fit_transform(training_set)
y_train = mlb.fit_transform(training_set[all_tiers_100].apply(util.array_labels, axis=1))

In [42]:
X_test = transformer.transform(testing_set)
y_test = mlb.fit_transform(testing_set[all_tiers_100].apply(util.array_labels, axis=1))

In [43]:
from skmultilearn.problem_transform import *
from sklearn.svm import SVC

In [45]:
from skmultilearn.adapt import *
from sklearn import metrics
def evaluate(clf):
    prediction = clf.fit(X_train, y_train).predict(X_test)
    print(f"{clf.get_params()}")
    print(f"Hamming loss: {metrics.hamming_loss(y_test,prediction)}")
    print(metrics.classification_report(y_test, prediction, target_names=mlb.classes_))

In [49]:
!pip install catboost

In [53]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from catboost import CatBoostClassifier
clf = OneVsRestClassifier(estimator=CatBoostClassifier(iterations=1000, 
                           task_type="GPU",
                           devices='0:1',random_state=1))

# BinaryRelevance(
#     classifier=RandomForestClassifier(n_jobs=-1),
#     require_dense=[True, True]
# )

In [54]:
evaluate(clf)

# verse 32d cpc = 
# micro avg       0.54      0.37      0.44      1505
# macro avg       0.41      0.24      0.27      1505
# weighted avg       0.48      0.37      0.39      1505
# samples avg       0.55      0.40      0.43      1505

#deepwalk 32d cpc = 
#                                     micro avg       0.54      0.37      0.44      1505
#                                     macro avg       0.44      0.24      0.28      1505
#                                  weighted avg       0.49      0.37      0.39      1505
#                                   samples avg       0.54      0.40      0.42      1505


#node2vec 16d cpc = 
#                                     micro avg       0.55      0.37      0.44      1505
#                                     macro avg       0.43      0.24      0.28      1505
#                                  weighted avg       0.49      0.37      0.39      1505
#                                   samples avg       0.56      0.41      0.43      1505

#******* node2vec 32d cpc =
#                                     micro avg       0.55      0.38      0.45      1505
#                                     macro avg       0.42      0.25      0.28      1505
#                                  weighted avg       0.49      0.38      0.40      1505
#                                   samples avg       0.56      0.41      0.43      1505

#node2vec 64d cpc = 
#                                     micro avg       0.55      0.37      0.44      1505
#                                     macro avg       0.45      0.24      0.28      1505
#                                  weighted avg       0.50      0.37      0.39      1505
#                                   samples avg       0.56      0.40      0.43      1505




Learning rate set to 0.033764
0:	learn: 0.6885213	total: 58.3ms	remaining: 58.3s
1:	learn: 0.6849968	total: 113ms	remaining: 56.4s
2:	learn: 0.6802531	total: 170ms	remaining: 56.5s
3:	learn: 0.6752729	total: 227ms	remaining: 56.5s
4:	learn: 0.6701230	total: 283ms	remaining: 56.3s
5:	learn: 0.6658506	total: 336ms	remaining: 55.7s
6:	learn: 0.6609763	total: 387ms	remaining: 54.9s
7:	learn: 0.6567945	total: 435ms	remaining: 53.9s
8:	learn: 0.6527024	total: 484ms	remaining: 53.3s
9:	learn: 0.6490880	total: 534ms	remaining: 52.9s
10:	learn: 0.6463715	total: 580ms	remaining: 52.1s
11:	learn: 0.6429027	total: 628ms	remaining: 51.7s
12:	learn: 0.6405320	total: 673ms	remaining: 51.1s
13:	learn: 0.6381160	total: 719ms	remaining: 50.7s
14:	learn: 0.6361246	total: 765ms	remaining: 50.2s
15:	learn: 0.6333692	total: 813ms	remaining: 50s
16:	learn: 0.6311053	total: 860ms	remaining: 49.7s
17:	learn: 0.6285760	total: 907ms	remaining: 49.5s
18:	learn: 0.6264388	total: 952ms	remaining: 49.1s
19:	learn: 0

163:	learn: 0.4792945	total: 7.78s	remaining: 39.7s
164:	learn: 0.4786176	total: 7.83s	remaining: 39.6s
165:	learn: 0.4780888	total: 7.88s	remaining: 39.6s
166:	learn: 0.4774907	total: 7.92s	remaining: 39.5s
167:	learn: 0.4768763	total: 7.97s	remaining: 39.5s
168:	learn: 0.4760764	total: 8.02s	remaining: 39.4s
169:	learn: 0.4758882	total: 8.06s	remaining: 39.4s
170:	learn: 0.4754515	total: 8.11s	remaining: 39.3s
171:	learn: 0.4745017	total: 8.16s	remaining: 39.3s
172:	learn: 0.4739564	total: 8.2s	remaining: 39.2s
173:	learn: 0.4735140	total: 8.25s	remaining: 39.2s
174:	learn: 0.4727585	total: 8.3s	remaining: 39.1s
175:	learn: 0.4719867	total: 8.34s	remaining: 39.1s
176:	learn: 0.4714374	total: 8.39s	remaining: 39s
177:	learn: 0.4706267	total: 8.44s	remaining: 39s
178:	learn: 0.4704688	total: 8.48s	remaining: 38.9s
179:	learn: 0.4696898	total: 8.53s	remaining: 38.9s
180:	learn: 0.4688750	total: 8.58s	remaining: 38.8s
181:	learn: 0.4681181	total: 8.63s	remaining: 38.8s
182:	learn: 0.4673

323:	learn: 0.3877949	total: 15.3s	remaining: 31.9s
324:	learn: 0.3876272	total: 15.3s	remaining: 31.8s
325:	learn: 0.3869192	total: 15.4s	remaining: 31.8s
326:	learn: 0.3859476	total: 15.4s	remaining: 31.7s
327:	learn: 0.3850824	total: 15.5s	remaining: 31.7s
328:	learn: 0.3849701	total: 15.5s	remaining: 31.6s
329:	learn: 0.3848348	total: 15.6s	remaining: 31.6s
330:	learn: 0.3842248	total: 15.6s	remaining: 31.5s
331:	learn: 0.3841897	total: 15.6s	remaining: 31.5s
332:	learn: 0.3834217	total: 15.7s	remaining: 31.4s
333:	learn: 0.3827069	total: 15.7s	remaining: 31.4s
334:	learn: 0.3826067	total: 15.8s	remaining: 31.3s
335:	learn: 0.3825497	total: 15.8s	remaining: 31.3s
336:	learn: 0.3816982	total: 15.9s	remaining: 31.2s
337:	learn: 0.3810708	total: 15.9s	remaining: 31.2s
338:	learn: 0.3802115	total: 16s	remaining: 31.2s
339:	learn: 0.3795989	total: 16s	remaining: 31.1s
340:	learn: 0.3794488	total: 16.1s	remaining: 31.1s
341:	learn: 0.3793700	total: 16.1s	remaining: 31s
342:	learn: 0.3787

483:	learn: 0.3084794	total: 22.9s	remaining: 24.4s
484:	learn: 0.3083820	total: 22.9s	remaining: 24.4s
485:	learn: 0.3079442	total: 23s	remaining: 24.3s
486:	learn: 0.3077038	total: 23s	remaining: 24.3s
487:	learn: 0.3071651	total: 23.1s	remaining: 24.2s
488:	learn: 0.3068452	total: 23.1s	remaining: 24.2s
489:	learn: 0.3063329	total: 23.2s	remaining: 24.1s
490:	learn: 0.3059483	total: 23.2s	remaining: 24.1s
491:	learn: 0.3056006	total: 23.3s	remaining: 24s
492:	learn: 0.3054949	total: 23.3s	remaining: 24s
493:	learn: 0.3049507	total: 23.4s	remaining: 23.9s
494:	learn: 0.3049038	total: 23.4s	remaining: 23.9s
495:	learn: 0.3048354	total: 23.5s	remaining: 23.8s
496:	learn: 0.3041764	total: 23.5s	remaining: 23.8s
497:	learn: 0.3034296	total: 23.6s	remaining: 23.7s
498:	learn: 0.3033119	total: 23.6s	remaining: 23.7s
499:	learn: 0.3029006	total: 23.7s	remaining: 23.7s
500:	learn: 0.3027926	total: 23.7s	remaining: 23.6s
501:	learn: 0.3024834	total: 23.7s	remaining: 23.6s
502:	learn: 0.301900

644:	learn: 0.2598072	total: 30.6s	remaining: 16.8s
645:	learn: 0.2597260	total: 30.6s	remaining: 16.8s
646:	learn: 0.2595025	total: 30.7s	remaining: 16.7s
647:	learn: 0.2594381	total: 30.7s	remaining: 16.7s
648:	learn: 0.2591819	total: 30.8s	remaining: 16.6s
649:	learn: 0.2589453	total: 30.8s	remaining: 16.6s
650:	learn: 0.2585894	total: 30.9s	remaining: 16.5s
651:	learn: 0.2582675	total: 30.9s	remaining: 16.5s
652:	learn: 0.2582144	total: 30.9s	remaining: 16.4s
653:	learn: 0.2579771	total: 31s	remaining: 16.4s
654:	learn: 0.2577513	total: 31s	remaining: 16.4s
655:	learn: 0.2572336	total: 31.1s	remaining: 16.3s
656:	learn: 0.2569684	total: 31.1s	remaining: 16.3s
657:	learn: 0.2565805	total: 31.2s	remaining: 16.2s
658:	learn: 0.2561838	total: 31.2s	remaining: 16.2s
659:	learn: 0.2557316	total: 31.3s	remaining: 16.1s
660:	learn: 0.2555393	total: 31.3s	remaining: 16.1s
661:	learn: 0.2554708	total: 31.4s	remaining: 16s
662:	learn: 0.2550081	total: 31.4s	remaining: 16s
663:	learn: 0.254376

...

414:	learn: 0.1631190	total: 20.1s	remaining: 28.4s
415:	learn: 0.1625722	total: 20.2s	remaining: 28.3s
416:	learn: 0.1623629	total: 20.2s	remaining: 28.3s
417:	learn: 0.1621109	total: 20.3s	remaining: 28.2s
418:	learn: 0.1620110	total: 20.3s	remaining: 28.2s
419:	learn: 0.1618722	total: 20.4s	remaining: 28.1s
420:	learn: 0.1616110	total: 20.4s	remaining: 28.1s
421:	learn: 0.1613184	total: 20.5s	remaining: 28s
422:	learn: 0.1610867	total: 20.5s	remaining: 28s
423:	learn: 0.1608813	total: 20.6s	remaining: 27.9s
424:	learn: 0.1607223	total: 20.6s	remaining: 27.9s
425:	learn: 0.1605002	total: 20.7s	remaining: 27.8s
426:	learn: 0.1597202	total: 20.7s	remaining: 27.8s
427:	learn: 0.1596133	total: 20.8s	remaining: 27.8s
428:	learn: 0.1592518	total: 20.8s	remaining: 27.7s
429:	learn: 0.1590882	total: 20.9s	remaining: 27.7s
430:	learn: 0.1589268	total: 20.9s	remaining: 27.6s
431:	learn: 0.1588088	total: 21s	remaining: 27.6s
432:	learn: 0.1586192	total: 21s	remaining: 27.5s
433:	learn: 0.158445

573:	learn: 0.1323065	total: 27.8s	remaining: 20.6s
574:	learn: 0.1318048	total: 27.8s	remaining: 20.6s
575:	learn: 0.1316540	total: 27.9s	remaining: 20.5s
576:	learn: 0.1312819	total: 27.9s	remaining: 20.5s
577:	learn: 0.1312168	total: 28s	remaining: 20.4s
578:	learn: 0.1309804	total: 28s	remaining: 20.4s
579:	learn: 0.1308970	total: 28.1s	remaining: 20.3s
580:	learn: 0.1308088	total: 28.1s	remaining: 20.3s
581:	learn: 0.1307043	total: 28.2s	remaining: 20.2s
582:	learn: 0.1306084	total: 28.2s	remaining: 20.2s
583:	learn: 0.1303439	total: 28.3s	remaining: 20.1s
584:	learn: 0.1302374	total: 28.3s	remaining: 20.1s
585:	learn: 0.1301122	total: 28.3s	remaining: 20s
586:	learn: 0.1300864	total: 28.4s	remaining: 20s
587:	learn: 0.1297469	total: 28.4s	remaining: 19.9s
588:	learn: 0.1296468	total: 28.5s	remaining: 19.9s
589:	learn: 0.1293631	total: 28.5s	remaining: 19.8s
590:	learn: 0.1293124	total: 28.6s	remaining: 19.8s
591:	learn: 0.1292755	total: 28.6s	remaining: 19.7s
592:	learn: 0.129224

733:	learn: 0.1112385	total: 35.5s	remaining: 12.9s
734:	learn: 0.1111306	total: 35.5s	remaining: 12.8s
735:	learn: 0.1109908	total: 35.6s	remaining: 12.8s
736:	learn: 0.1107418	total: 35.6s	remaining: 12.7s
737:	learn: 0.1103814	total: 35.7s	remaining: 12.7s
738:	learn: 0.1103227	total: 35.7s	remaining: 12.6s
739:	learn: 0.1101907	total: 35.8s	remaining: 12.6s
740:	learn: 0.1101524	total: 35.8s	remaining: 12.5s
741:	learn: 0.1098165	total: 35.9s	remaining: 12.5s
742:	learn: 0.1097879	total: 35.9s	remaining: 12.4s
743:	learn: 0.1097180	total: 36s	remaining: 12.4s
744:	learn: 0.1096387	total: 36s	remaining: 12.3s
745:	learn: 0.1095476	total: 36.1s	remaining: 12.3s
746:	learn: 0.1093020	total: 36.1s	remaining: 12.2s
747:	learn: 0.1092366	total: 36.2s	remaining: 12.2s
748:	learn: 0.1091789	total: 36.2s	remaining: 12.1s
749:	learn: 0.1090662	total: 36.3s	remaining: 12.1s
750:	learn: 0.1089897	total: 36.3s	remaining: 12s
751:	learn: 0.1088797	total: 36.4s	remaining: 12s
752:	learn: 0.108825

894:	learn: 0.0928728	total: 43.4s	remaining: 5.09s
895:	learn: 0.0928445	total: 43.5s	remaining: 5.05s
896:	learn: 0.0926078	total: 43.5s	remaining: 5s
897:	learn: 0.0925705	total: 43.6s	remaining: 4.95s
898:	learn: 0.0924260	total: 43.6s	remaining: 4.9s
899:	learn: 0.0924166	total: 43.7s	remaining: 4.85s
900:	learn: 0.0923084	total: 43.7s	remaining: 4.8s
901:	learn: 0.0922532	total: 43.8s	remaining: 4.76s
902:	learn: 0.0921946	total: 43.8s	remaining: 4.71s
903:	learn: 0.0920168	total: 43.9s	remaining: 4.66s
904:	learn: 0.0920055	total: 43.9s	remaining: 4.61s
905:	learn: 0.0919473	total: 44s	remaining: 4.56s
906:	learn: 0.0917405	total: 44s	remaining: 4.52s
907:	learn: 0.0914210	total: 44.1s	remaining: 4.47s
908:	learn: 0.0914055	total: 44.1s	remaining: 4.42s
909:	learn: 0.0913876	total: 44.2s	remaining: 4.37s
910:	learn: 0.0913287	total: 44.2s	remaining: 4.32s
911:	learn: 0.0912785	total: 44.3s	remaining: 4.27s
912:	learn: 0.0912250	total: 44.3s	remaining: 4.22s
913:	learn: 0.0910217

/home/martin/anaconda3/envs/phenetics2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [84]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

parameters = [
    {
        'classifier': [MultinomialNB()],
        'classifier__alpha': [0.7, 1.0],
    },
    {
        'classifier': [SVC()],
        'classifier__kernel': ['rbf', 'linear'],
    },
]

clf = GridSearchCV(ClassifierChain(), parameters, scoring='accuracy')
clf.fit(X_train, y_train)

print (clf.best_params_, clf.best_score_)



{'classifier': MultinomialNB(alpha=0.7), 'classifier__alpha': 0.7} 0.061723499867829754


In [150]:
import lightgbm as lgb
evaluate(BinaryRelevance(lgb.LGBMClassifier()))

{'classifier': LGBMClassifier(), 'classifier__boosting_type': 'gbdt', 'classifier__class_weight': None, 'classifier__colsample_bytree': 1.0, 'classifier__importance_type': 'split', 'classifier__learning_rate': 0.1, 'classifier__max_depth': -1, 'classifier__min_child_samples': 20, 'classifier__min_child_weight': 0.001, 'classifier__min_split_gain': 0.0, 'classifier__n_estimators': 100, 'classifier__n_jobs': -1, 'classifier__num_leaves': 31, 'classifier__objective': None, 'classifier__random_state': None, 'classifier__reg_alpha': 0.0, 'classifier__reg_lambda': 0.0, 'classifier__silent': True, 'classifier__subsample': 1.0, 'classifier__subsample_for_bin': 200000, 'classifier__subsample_freq': 0, 'require_dense': [True, True]}
Hamming loss: 0.2774036662925552
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.42      0.33      0.37        84
           Analysis and Modeling: 3D Modeling       0.38   

/home/martin/anaconda3/envs/phenetics2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/martin/anaconda3/envs/phenetics2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [149]:
evaluate(MLkNN())

/home/martin/anaconda3/envs/phenetics2/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_neighbors=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


{'k': 10, 's': 1.0, 'ignore_first_neighbours': 0}
Hamming loss: 0.2826412270856715
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.30      0.12      0.17        84
           Analysis and Modeling: 3D Modeling       0.40      0.08      0.14        71
                            Anatomical Target       0.66      0.79      0.72       164
           Anatomical Target: Lower Extremity       0.45      0.31      0.37       113
     Anatomical Target: Lower Extremity - Hip       0.40      0.05      0.09        40
    Anatomical Target: Lower Extremity - Knee       0.35      0.07      0.12        82
                     Anatomical Target: Torso       0.00      0.00      0.00        35
             Anatomical Target: Torso - Spine       0.00      0.00      0.00        21
           Anatomical Target: Upper Extremity       0.30      0.10      0.15        31
Anatomical Target: Upper Extremity - Shoulder 

/home/martin/anaconda3/envs/phenetics2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/martin/anaconda3/envs/phenetics2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [69]:
evaluate(LabelPowerset(
    classifier=SVC(),
    require_dense=[False, True]
))

{'classifier': SVC(), 'classifier__C': 1.0, 'classifier__break_ties': False, 'classifier__cache_size': 200, 'classifier__class_weight': None, 'classifier__coef0': 0.0, 'classifier__decision_function_shape': 'ovr', 'classifier__degree': 3, 'classifier__gamma': 'scale', 'classifier__kernel': 'rbf', 'classifier__max_iter': -1, 'classifier__probability': False, 'classifier__random_state': None, 'classifier__shrinking': True, 'classifier__tol': 0.001, 'classifier__verbose': False, 'require_dense': [False, True]}
Hamming loss: 0.3887018331462776
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.00      0.00      0.00        84
           Analysis and Modeling: 3D Modeling       0.00      0.00      0.00        71
                            Anatomical Target       0.74      0.10      0.18       164
           Anatomical Target: Lower Extremity       0.61      0.12      0.21       113
     Anatomical Ta

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.feature_selection import f_classif


In [ ]:
pipe = Pipeline(steps=[('transformer', transformer),
                       ('feature_selection', SelectFromModel(LinearSVC(penalty="l1"))),
                       #('svd', TruncatedSVD(random_state=42)),
                       #('dummy', OneVsRestClassifier(DummyClassifier()))
                       #('svc', OneVsRestClassifier(SVC(random_state=42), n_jobs=-1))
                       ('rf',  RandomForestClassifier(n_jobs=-1, random_state=42))
                       #('lr', OneVsRestClassifier(LogisticRegression(n_jobs=-1), n_jobs=-1))
                       #('cat', OneVsRestClassifier(CatBoostClassifier(verbose=True)))
                       #('knn', KNeighborsClassifier(n_jobs=-1))
                      ], 
                verbose=True,
                memory="cachedir/")

In [ ]:
training_labels[:,0]

In [ ]:
from sklearn.feature_selection import chi2
selected_features = [] 
for label in range(0, len(subset)):
    selector = SelectKBest(chi2, k='all')
    selector.fit(X, training_labels.values[:,label])
    selected_features.append(list(selector.scores_))

In [ ]:
selected_features = np.nan_to_num(np.array(selected_features))

In [ ]:
selected_features

In [ ]:
selected_features.mean(axis=0).mean()

In [ ]:
features_set = np.mean(np.nan_to_num(np.array(selected_features)), axis=0) > 0.8

In [ ]:
features_set.astype(int).sum()

In [ ]:
X.shape

In [ ]:
feature_subset = X[:,features_set]

In [ ]:
feature_subset.shape

In [ ]:
model = RandomForestClassifier()
model.fit(X[:,features_set], training_labels)
predictions = model.predict(Xtest[:,features_set])
print(classification_report(testing_labels, predictions, target_names=subset))

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt import *
from skopt.space import Real, Categorical, Integer

param_grid = {
    #'rf__bootstrap': [True, False],
    'max_depth': (10, 1000), #[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 125, 150, 175, 200, 250, 300, 350, 400, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': (1, 12),
 'min_samples_split': (2, 12),
 'n_estimators': (5, 1000)          
             }


# param_grid = {
#     'svd__n_components': Integer(64,10000),
#     'svc__estimator__C': Real(1e-6, 1e+6, prior='log-uniform'),
#     'svc__estimator__gamma': Real(1e-6, 1e+6, prior='log-uniform'),
#     'svc__estimator__degree': Integer(1,8),
#     'svc__estimator__kernel': Categorical(['linear', 'poly', 'rbf']),
# }

# param_grid = {
# #  'svd__n_components': np.arange(64, 5000, 100),
#  'knn__leaf_size': np.arange(1, 50, 1),
#  'knn__metric': ['minkowski', 'euclidean'],
#  'knn__n_neighbors': [2,3,4,5,6,7,8,9,10,11,12],
#  'knn__weights': ['distance', 'uniform']
# }
search = BayesSearchCV(model, param_grid, n_iter=50, n_points=3, pre_dispatch=36, refit=True, cv=3, verbose=10, random_state=42, n_jobs=-1)

from tqdm.auto import tqdm
from tqdm.utils import CallbackIOWrapper

with tqdm(total=search.total_iterations) as pbar:
    def on_step(optim_result):
        print(optim_result)
        pbar.update(9)
        return False
    search.fit(X[:,features_set], training_labels, callback=on_step)


In [ ]:
f1_score(testing_labels, predictions, average="weighted")

In [ ]:
hamming_loss(testing_labels, predictions)

In [ ]:
plot_confusion_matrix(pipe, testing_set, testing_labels)

In [ ]:
training_predictions = search.best_estimator_.predict(X[:,features_set])
print(classification_report(training_labels, training_predictions, target_names=subset))

In [ ]:
from skopt import BayesSearchCV
from skopt import *
from skopt.space import Real, Categorical, Integer

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
param_grid = {
    #'rf__bootstrap': [True, False],
    'rf__max_depth': (10, 1000), #[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 125, 150, 175, 200, 250, 300, 350, 400, None],
 #'rf__max_features': ['auto', 'sqrt'],
 'rf__min_samples_leaf': (1, 12),
 'rf__min_samples_split': (2, 12),
 'rf__n_estimators': (5, 1000)          
             }


# param_grid = {
#     'svd__n_components': Integer(64,10000),
#     'svc__estimator__C': Real(1e-6, 1e+6, prior='log-uniform'),
#     'svc__estimator__gamma': Real(1e-6, 1e+6, prior='log-uniform'),
#     'svc__estimator__degree': Integer(1,8),
#     'svc__estimator__kernel': Categorical(['linear', 'poly', 'rbf']),
# }

# param_grid = {
# #  'svd__n_components': np.arange(64, 5000, 100),
#  'knn__leaf_size': np.arange(1, 50, 1),
#  'knn__metric': ['minkowski', 'euclidean'],
#  'knn__n_neighbors': [2,3,4,5,6,7,8,9,10,11,12],
#  'knn__weights': ['distance', 'uniform']
# }
search = BayesSearchCV(pipe, param_grid, n_iter=50, n_points=3, pre_dispatch=36, refit=True, cv=3, verbose=10, random_state=42, n_jobs=-1)

from tqdm.auto import tqdm
from tqdm.utils import CallbackIOWrapper

with tqdm(total=search.total_iterations) as pbar:
    def on_step(optim_result):
        print(optim_result)
        pbar.update(9)
        return False
    search.fit(training_set, training_labels, callback=on_step)



In [ ]:
print(search.best_estimator_.get_params())

In [ ]:
from sklearn.metrics import *
#pipe.fit(training_set, training_labels)
#predictions = pipe.predict(testing_set)
predictions = search.best_estimator_.predict(Xtest[:,features_set])
print(classification_report(testing_labels, predictions, target_names=subset))

In [ ]:
from sklearn.metrics import *
#pipe.fit(training_set, training_labels)
#predictions = pipe.predict(testing_set)
predictions = search.best_estimator_.predict(testing_set)
print(classification_report(testing_labels, predictions, target_names=all_tiers_100))

In [ ]:
from sklearn.metrics import *
#pipe.fit(training_set, training_labels)
predictions = pipe.predict(testing_set)
print(classification_report(testing_labels, predictions, target_names=all_tiers_100))

In [ ]:
predictions = predictions = search.best_estimator_.predict(training_set)
print(classification_report(training_labels, predictions, target_names=subset))


#  Guessing Baseline

                                      precision    recall  f1-score   support

                 AnalysisAndModeling       0.31      0.32      0.32        84
                    AnatomicalTarget       0.63      0.63      0.63       164
                             Imaging       0.57      0.56      0.56       133
                       Manufacturing       0.38      0.48      0.43        83
                  SpecificationofUse       0.33      0.33      0.33        79
                      SurgicalMethod       0.19      0.20      0.20        40
      AnalysisAndModeling_3DModeling       0.22      0.21      0.22        71
     AnatomicalTarget_LowerExtremity       0.45      0.43      0.44       113
              AnatomicalTarget_Torso       0.19      0.17      0.18        35
     AnatomicalTarget_UpperExtremity       0.26      0.26      0.26        31
                          Imaging_CT       0.14      0.19      0.16        59
                         Imaging_MRI       0.24      0.20      0.22        59
                  Imaging_Ultrasound       0.17      0.19      0.18        32
 Manufacturing_AdditiveManufacturing       0.24      0.24      0.24        38
       PersonalizedProduct_Guide/Jig       0.55      0.46      0.50       120
         PersonalizedProduct_Implant       0.49      0.53      0.51       124
          SpecificationofUse_Disease       0.19      0.20      0.20        30
 SpecificationofUse_JointReplacement       0.14      0.23      0.17        44
 AnatomicalTarget_LowerExtremity_Hip       0.21      0.17      0.19        40
AnatomicalTarget_LowerExtremity_Knee       0.32      0.34      0.33        82

                           micro avg       0.38      0.39      0.38      1461
                           macro avg       0.31      0.32      0.31      1461
                        weighted avg       0.38      0.39      0.39      1461
                         samples avg       0.38      0.40      0.37      1461

# RF

 {'rf__bootstrap': False,
 'rf__ccp_alpha': 0.0,
 'rf__class_weight': None,
 'rf__criterion': 'gini',
 'rf__max_depth': 150,
 'rf__max_features': 'sqrt',
 'rf__max_leaf_nodes': None,
 'rf__max_samples': None,
 'rf__min_impurity_decrease': 0.0,
 'rf__min_impurity_split': None,
 'rf__min_samples_leaf': 1,
 'rf__min_samples_split': 2,
 'rf__min_weight_fraction_leaf': 0.0,
 'rf__n_estimators': 200,
 'rf__n_jobs': None,
 'rf__oob_score': False,
 'rf__random_state': None,
 'rf__verbose': 0,
 'rf__warm_start': False}

                                      precision    recall  f1-score   support

                 AnalysisAndModeling       0.43      0.24      0.31        84
                    AnatomicalTarget       0.70      0.78      0.74       164
                             Imaging       0.60      0.59      0.60       133
                       Manufacturing       0.37      0.25      0.30        83
                  SpecificationofUse       0.42      0.32      0.36        79
                      SurgicalMethod       0.71      0.30      0.42        40
      AnalysisAndModeling_3DModeling       0.38      0.18      0.25        71
     AnatomicalTarget_LowerExtremity       0.53      0.46      0.49       113
              AnatomicalTarget_Torso       0.08      0.03      0.04        35
     AnatomicalTarget_UpperExtremity       0.11      0.03      0.05        31
                          Imaging_CT       0.18      0.10      0.13        59
                         Imaging_MRI       0.28      0.14      0.18        59
                  Imaging_Ultrasound       0.00      0.00      0.00        32
 Manufacturing_AdditiveManufacturing       0.23      0.08      0.12        38
       PersonalizedProduct_Guide/Jig       0.64      0.41      0.50       120
         PersonalizedProduct_Implant       0.59      0.74      0.66       124
          SpecificationofUse_Disease       0.06      0.03      0.04        30
 SpecificationofUse_JointReplacement       0.21      0.16      0.18        44
 AnatomicalTarget_LowerExtremity_Hip       0.26      0.12      0.17        40
AnatomicalTarget_LowerExtremity_Knee       0.47      0.33      0.39        82

                           micro avg       0.50      0.38      0.43      1461
                           macro avg       0.36      0.26      0.30      1461
                        weighted avg       0.45      0.38      0.40      1461
                         samples avg       0.53      0.41      0.43      1461



# KNN

 'knn__algorithm': 'auto',
 'knn__leaf_size': 30,
 'knn__metric': 'euclidean',
 'knn__metric_params': None,
 'knn__n_jobs': None,
 'knn__n_neighbors': 2,
 'knn__p': 2,
 'knn__weights': 'distance'
                                      precision    recall  f1-score   support

                 AnalysisAndModeling       0.38      0.43      0.40        84
                    AnatomicalTarget       0.71      0.60      0.65       164
                             Imaging       0.63      0.62      0.62       133
                       Manufacturing       0.34      0.37      0.36        83
                  SpecificationofUse       0.34      0.47      0.39        79
                      SurgicalMethod       0.39      0.30      0.34        40
      AnalysisAndModeling_3DModeling       0.35      0.38      0.36        71
     AnatomicalTarget_LowerExtremity       0.60      0.50      0.54       113
              AnatomicalTarget_Torso       0.24      0.11      0.15        35
     AnatomicalTarget_UpperExtremity       0.16      0.16      0.16        31
                          Imaging_CT       0.24      0.32      0.27        59
                         Imaging_MRI       0.24      0.32      0.28        59
                  Imaging_Ultrasound       0.16      0.28      0.21        32
 Manufacturing_AdditiveManufacturing       0.18      0.13      0.15        38
       PersonalizedProduct_Guide/Jig       0.59      0.39      0.47       120
         PersonalizedProduct_Implant       0.56      0.76      0.64       124
          SpecificationofUse_Disease       0.19      0.33      0.24        30
 SpecificationofUse_JointReplacement       0.12      0.14      0.13        44
 AnatomicalTarget_LowerExtremity_Hip       0.33      0.30      0.32        40
AnatomicalTarget_LowerExtremity_Knee       0.48      0.39      0.43        82

                           micro avg       0.43      0.44      0.43      1461
                           macro avg       0.36      0.37      0.36      1461
                        weighted avg       0.45      0.44      0.44      1461
                         samples avg       0.45      0.46      0.41      1461

# KNN + SVD

 precision    recall  f1-score   support

                 AnalysisAndModeling       0.38      0.43      0.40        84
                    AnatomicalTarget       0.71      0.60      0.65       164
                             Imaging       0.63      0.62      0.62       133
                       Manufacturing       0.34      0.37      0.36        83
                  SpecificationofUse       0.34      0.47      0.39        79
                      SurgicalMethod       0.39      0.30      0.34        40
      AnalysisAndModeling_3DModeling       0.35      0.38      0.36        71
     AnatomicalTarget_LowerExtremity       0.60      0.50      0.54       113
              AnatomicalTarget_Torso       0.24      0.11      0.15        35
     AnatomicalTarget_UpperExtremity       0.16      0.16      0.16        31
                          Imaging_CT       0.24      0.32      0.27        59
                         Imaging_MRI       0.24      0.32      0.28        59
                  Imaging_Ultrasound       0.16      0.28      0.21        32
 Manufacturing_AdditiveManufacturing       0.18      0.13      0.15        38
       PersonalizedProduct_Guide/Jig       0.59      0.39      0.47       120
         PersonalizedProduct_Implant       0.56      0.76      0.64       124
          SpecificationofUse_Disease       0.19      0.33      0.24        30
 SpecificationofUse_JointReplacement       0.12      0.14      0.13        44
 AnatomicalTarget_LowerExtremity_Hip       0.33      0.30      0.32        40
AnatomicalTarget_LowerExtremity_Knee       0.48      0.39      0.43        82

                           micro avg       0.43      0.44      0.43      1461
                           macro avg       0.36      0.37      0.36      1461
                        weighted avg       0.45      0.44      0.44      1461
                         samples avg       0.45      0.46      0.41      1461

'svd__algorithm': 'randomized',
 'svd__n_components': 2564,
 'svd__n_iter': 5,
 'svd__random_state': 42,
 'svd__tol': 0.0,
 'knn__algorithm': 'auto',
 'knn__leaf_size': 30,
 'knn__metric': 'euclidean',
 'knn__metric_params': None,
 'knn__n_jobs': None,
 'knn__n_neighbors': 2,
 'knn__p': 2,
 'knn__weights': 'distance'
